# <img src="https://upload.wikimedia.org/wikipedia/commons/6/60/NISAR_artist_concept.jpg" width=400 align="left"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/NISAR_Mission_Logo.png" width=400 align="left"/><br><br><br><br><br>



# Convert ALOS2 Data to GCOV

In preparation for NISAR, this notebook converts ALOS-2 SLC data to NISAR RSLC to NISAR GCOV.


These steps can be run in the command line, details are below.

In [1]:
import os
import sys
from pathlib import Path
import subprocess
import fnmatch
import zipfile
import h5py
from string import Template
from osgeo import gdal


Choose your place to save your outputs. 

Scratch can be used for short-term storage, but the cost is much higher. Files for long-term storage should be moved to an S3 bucket.

In [2]:
scratch_folder = Path('/scratch/alex_eco_test/') ##change to your folder


Choose your AOI

New directories will be made to store files for this AOI

In [4]:
aoi = 'waxlakedelta'

aoi_dir = scratch_folder/aoi
RSLC_dir = aoi_dir / 'RSLC'
ALOS2_dir = aoi_dir / 'ALOS2'
DEM_dir = aoi_dir / 'DEM'
GCOV_dir = aoi_dir / 'GCOV'
TMP_dir = aoi_dir / 'TMP'

Path(aoi_dir).mkdir(parents=True, exist_ok=True)
Path(RSLC_dir).mkdir(parents=True, exist_ok=True)
Path(ALOS2_dir).mkdir(parents=True, exist_ok=True)
Path(DEM_dir).mkdir(parents=True, exist_ok=True)
Path(GCOV_dir).mkdir(parents=True, exist_ok=True)
Path(TMP_dir).mkdir(parents=True, exist_ok=True)



## Copy the ALOS2 URLs into a list

In [5]:
list_of_ALOS_SLCs = ['https://cumulus.asf.alaska.edu/L1.1/A4/0000436588_001001_ALOS2390510580-210816.zip',
                        'https://cumulus.asf.alaska.edu/L1.1/A4/0000470239_001001_ALOS2427770580-220425.zip']
                        

## Download the ALOS2 files using wget

In [6]:
for file in list_of_ALOS_SLCs:
    print(file)
    filename = ALOS2_dir / file.split('/')[-1]
    if os.path.isfile(filename)==False:
        command = f"wget -P {ALOS2_dir} -q {file}"
        output = subprocess.check_output(command, shell=True)


https://cumulus.asf.alaska.edu/L1.1/A4/0000436588_001001_ALOS2390510580-210816.zip
https://cumulus.asf.alaska.edu/L1.1/A4/0000470239_001001_ALOS2427770580-220425.zip


## Unzip the ALOS files

If you don't want to keep the zip files, you should uncomment the os.remove line

In [7]:
zipfiles = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(files,'*.zip')]
for file in zipfiles:
    print(file)
    if os.path.isdir(ALOS2_dir / file[:-4]):
        ##check full unzipping
        if sum(os.path.getsize(ALOS2_dir / file[:-4] / f) for f in os.listdir(ALOS2_dir / file[:-4]) if os.path.isfile(ALOS2_dir / file[:-4] /f)) < 6509714637:
            with zipfile.ZipFile(file, 'r') as zip_ref:
                zip_ref.extractall(ALOS2_dir/file.split('/')[-1][:-4])
            # os.remove(file)
    else:
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(ALOS2_dir/file.split('/')[-1][:-4])


/scratch/alex_eco_test/waxlakedelta/ALOS2/0000436588_001001_ALOS2390510580-210816.zip
/scratch/alex_eco_test/waxlakedelta/ALOS2/0000470239_001001_ALOS2427770580-220425.zip


## Get a list of folders for each ALOS2 file

In [8]:
ALOS2folders = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(dirnames,'*')]
ALOS2folders

['/scratch/alex_eco_test/waxlakedelta/ALOS2/0000470239_001001_ALOS2427770580-220425',
 '/scratch/alex_eco_test/waxlakedelta/ALOS2/0000436588_001001_ALOS2390510580-210816']

## Convert ALOS2 SLC to NISAR RSLC format

In [14]:
for ALOS2folder in ALOS2folders[:]:
    ALOS2_id = ALOS2folder.split('/')[-1]
    print('')
    print(ALOS2_id)
    
    ##ALOS2 SLC --> NISAR RSLC
    command = f"conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i {ALOS2folder} -o {RSLC_dir/ALOS2_id}.h5"
    print('')
    print(command)    
    if os.path.isfile(f"{RSLC_dir/ALOS2_id}.h5")==False: #or os.path.getsize(f"{RSLC_dir/ALOS2_id}.h5")<6000000000:
        try: os.remove(f"{RSLC_dir/ALOS2_id}.h5")
        except:''
        output = subprocess.check_output(command, shell=True)


0000470239_001001_ALOS2427770580-220425

conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i /scratch/alex_eco_test/waxlakedelta/ALOS2/0000470239_001001_ALOS2427770580-220425 -o /scratch/alex_eco_test/waxlakedelta/RSLC/0000470239_001001_ALOS2427770580-220425.h5

0000436588_001001_ALOS2390510580-210816

conda run -n isce3_src /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i /scratch/alex_eco_test/waxlakedelta/ALOS2/0000436588_001001_ALOS2390510580-210816 -o /scratch/alex_eco_test/waxlakedelta/RSLC/0000436588_001001_ALOS2390510580-210816.h5


In [15]:
ALOS2_RSLCs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(ALOS2_dir)
                for f in fnmatch.filter(files,'*')]


## Get a DEM for the first ALOS2 file in the stack 

In [16]:
for ALOS2_RSLC in ALOS2_RSLCs[:]:
    ALOS2_id = ALOS2_RSLC.split('/')[-1][:-3]
    print('')
    print(ALOS2_id)

    ## Get NISAR DEM
    command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p {ALOS2_dir/ALOS2_id}.h5 -o {DEM_dir/ALOS2_id}.vrt"
    print('')
    print(command)    

    if os.path.isfile(f"{DEM_dir/ALOS2_id}.vrt")==False:
        output = subprocess.check_output(command, shell=True)


0000436588_001001_ALOS2390510580-210816.

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/alex_eco_test/waxlakedelta/ALOS2/0000436588_001001_ALOS2390510580-210816..h5 -o /scratch/alex_eco_test/waxlakedelta/DEM/0000436588_001001_ALOS2390510580-210816..vrt


overwriting variable {'LD_LIBRARY_PATH'}
Traceback (most recent call last):
  File "/home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py", line 493, in <module>
    main(opts)
  File "/home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py", line 461, in main
    poly = determine_polygon(opts.product, opts.bbox)
  File "/home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py", line 126, in determine_polygon
    poly = get_geo_polygon(ref_slc)
  File "/home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py", line 185, in get_geo_polygon
    productSlc = SLC(hdf5file=ref_slc)
  File "/home/jovyan/isce3/build/packages/pyre/components/Actor.py", line 169, in __call__
    instance = super().__call__(name=name, locator=locator, implicit=implicit, **kwds)
  File "/home/jovyan/isce3/build/packages/nisar/products/readers/SLC/SLC.py", line 33, in __init__
    super().__init__(**kwds)
  File "/home/jovyan/isce3/build/packages/nisar/products/readers/Base/Base.py"

CalledProcessError: Command 'conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/alex_eco_test/waxlakedelta/ALOS2/0000436588_001001_ALOS2390510580-210816..h5 -o /scratch/alex_eco_test/waxlakedelta/DEM/0000436588_001001_ALOS2390510580-210816..vrt' returned non-zero exit status 1.

## Process NISAR RSLC to GCOV

In [ ]:
src = gdal.Open(f"{DEM_dir/ALOS2_id}.vrt")
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
lrx = ulx + (src.RasterXSize * xres)
lry = uly + (src.RasterYSize * yres)

x1,y1,x2,y2 = math.floor(ulx),math.floor(uly),math.floor(lrx),math.floor(lry)


zone = int(np.ceil((ulx + 180)/6))

if y1>=0:
    EPSG = 32600+zone
elif y1<0
    NS = 's'
    EPSG = 32600+zone

In [22]:
for ALOS2_RSLC in ALOS2_RSLCs[:]:
    ALOS2_id = ALOS2_RSLC.split('/')[-1][:-3]
    print('')
    print(ALOS2_id)

  
    ## NISAR RSLC --> NISAR GCOV
    filein = open('gcov_template.yaml')
    template = Template(filein.read())
    replacements = {'inputfile':  f"{ALOS2_dir/ALOS2_id}.h5",
                    'outputfile':  f"{GCOV_dir/ALOS2_id}_gcov_utm.h5",
                    'demfile': f"{DEM_dir/ALOS2_id}.vrt",
                    'tmp': f"{TMP_dir}/",
                    'epsg': 32615,
                    'xposting': 20,
                    'yposting': 20,
                    }
    makeoutput = template.substitute(replacements)
    file = open('%s/%s.yaml' %(GCOV_dir,ALOS2_id),'w')
    file.write(makeoutput)
    file.close()
    filein.close()

    command = f"conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py {GCOV_dir/ALOS2_id}.yaml"
    print('')
    print(command)
    if os.path.isfile(f"{GCOV_dir/ALOS2_id}_gcov_utm.h5")==False:
        output = subprocess.check_output(command, shell=True)



0000509061_001001_ALOS2479520580-230410

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/alex_eco_test/waxlakedelta/ALOS2/0000509061_001001_ALOS2479520580-230410.h5 -o /scratch/alex_eco_test/waxlakedelta/DEM/0000509061_001001_ALOS2479520580-230410.vrt

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py /scratch/alex_eco_test/waxlakedelta/GCOV/0000509061_001001_ALOS2479520580-230410.yaml

0000528515_001001_ALOS2500220580-230828

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/alex_eco_test/waxlakedelta/ALOS2/0000528515_001001_ALOS2500220580-230828.h5 -o /scratch/alex_eco_test/waxlakedelta/DEM/0000528515_001001_ALOS2500220580-230828.vrt

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py /scratch/alex_eco_test/waxlakedelta/GCOV/0000528515_001001_ALOS2500220580-230828.yaml

0000436588_001001_ALOS2390510580-210816

conda run -n

overwriting variable {'LD_LIBRARY_PATH'}
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:207: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  h5py.File(output_hdf5, 'w', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:489: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'a', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:765: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.create('_FillValue', data=np.nan)
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:769: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.crea


0000470239_001001_ALOS2427770580-220425

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/alex_eco_test/waxlakedelta/ALOS2/0000470239_001001_ALOS2427770580-220425.h5 -o /scratch/alex_eco_test/waxlakedelta/DEM/0000470239_001001_ALOS2427770580-220425.vrt

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py /scratch/alex_eco_test/waxlakedelta/GCOV/0000470239_001001_ALOS2427770580-220425.yaml


overwriting variable {'LD_LIBRARY_PATH'}
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:207: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  h5py.File(output_hdf5, 'w', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:489: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'a', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:765: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.create('_FillValue', data=np.nan)
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:769: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.crea


0000477810_001001_ALOS2444330580-220815

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/alex_eco_test/waxlakedelta/ALOS2/0000477810_001001_ALOS2444330580-220815.h5 -o /scratch/alex_eco_test/waxlakedelta/DEM/0000477810_001001_ALOS2444330580-220815.vrt

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py /scratch/alex_eco_test/waxlakedelta/GCOV/0000477810_001001_ALOS2444330580-220815.yaml


...

overwriting variable {'LD_LIBRARY_PATH'}
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:207: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  h5py.File(output_hdf5, 'w', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:489: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'a', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:765: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.create('_FillValue', data=np.nan)
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:769: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.crea


0000510737_001001_ALOS2481590580-230424

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/alex_eco_test/waxlakedelta/ALOS2/0000510737_001001_ALOS2481590580-230424.h5 -o /scratch/alex_eco_test/waxlakedelta/DEM/0000510737_001001_ALOS2481590580-230424.vrt

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py /scratch/alex_eco_test/waxlakedelta/GCOV/0000510737_001001_ALOS2481590580-230424.yaml


overwriting variable {'LD_LIBRARY_PATH'}
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:207: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  h5py.File(output_hdf5, 'w', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:489: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'a', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:765: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.create('_FillValue', data=np.nan)
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:769: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.crea


0000526924_001001_ALOS2498150580-230814

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p /scratch/alex_eco_test/waxlakedelta/ALOS2/0000526924_001001_ALOS2498150580-230814.h5 -o /scratch/alex_eco_test/waxlakedelta/DEM/0000526924_001001_ALOS2498150580-230814.vrt

conda run -n isce3_src /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py /scratch/alex_eco_test/waxlakedelta/GCOV/0000526924_001001_ALOS2498150580-230814.yaml


overwriting variable {'LD_LIBRARY_PATH'}
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:207: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  h5py.File(output_hdf5, 'w', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/build/packages/nisar/workflows/h5_prep.py:489: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(output_hdf5, 'a', libver='latest', swmr=True) as dst_h5:
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:765: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.create('_FillValue', data=np.nan)
/home/jovyan/isce3/python/packages/nisar/workflows/gcov.py:769: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  dset.attrs.crea

See final list of NISAR GCOV data files

In [ ]:
NISAR_GCOVs = [os.path.join(dirpath,f)
                for dirpath,dirnames, files in os.walk(GCOV_dir)
                for f in fnmatch.filter(files,'*.h5')]
NISAR_GCOVs

In [ ]:
## Command line option

######## ALOS2 SLC to NISAR RSLC
# for f in /scratch/alex_eco_test/ALOS2/southfork/*/; 
# do python /home/jovyan/isce3/share/nisar/examples/alos2_to_nisar_l1.py -i $f -o "${f%.*}".h5; 
# done

######## Get NISAR DEM
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/stage_dem.py -p $f -o "${f%.*}".vrt; 
# done

######## NISAR RSLC to NISAR GCOV
# for f in /scratch/alex_eco_test/ALOS2/southfork/rslc/*.yaml; 
# do python /home/jovyan/isce3/python/packages/nisar/workflows/gcov.py $f; 
# done